In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.preprocessing import RobustScaler # ALTERADO: De StandardScaler para RobustScaler
import os
import sys
import datetime
import joblib

In [2]:
BASE_DRIVE_PATH = "/content/drive/MyDrive"

TRAINING_DATA_FILE = os.path.join(BASE_DRIVE_PATH, "training_data.csv")

DNABERT_EMBEDDINGS_FILE = os.path.join(BASE_DRIVE_PATH, "dnabert_embeddings.npy")

NORMALIZATION_SCALER_FILE = os.path.join(BASE_DRIVE_PATH, "scaler_robust.pkl")

RESULTS_OUTPUT_DIR = BASE_DRIVE_PATH
os.makedirs(RESULTS_OUTPUT_DIR, exist_ok=True)

RESULTS_FILE_PREFIX = "fnn_robust_results"
PREDICTIONS_DATA_PREFIX = "fnn_robust_predictions_data"

In [3]:
# --- 2. Configurações da FNN ---
FNN_HIDDEN_DIM = 256
FNN_LEARNING_RATE = 0.001
FNN_NUM_EPOCHS = 100
FNN_BATCH_SIZE = 64

In [4]:
# --- 3. Carregar Dados Originais e Embeddings Pré-Calculados ---
print("\n--- A carregar dados e embeddings pré-calculados ---")

try:
    df = pd.read_csv(TRAINING_DATA_FILE)
    print(f"Dados originais carregados com sucesso de {TRAINING_DATA_FILE}. Dimensão: {df.shape}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro de dados original '{TRAINING_DATA_FILE}' não foi encontrado.")
    sys.exit(1)

if not all(col in df.columns for col in ['target', 'sequence', 'prot_scaled']):
    print("ERRO: Colunas inesperadas no DataFrame original")
    sys.exit(1)

y_scaled = torch.tensor(df['prot_scaled'].values, dtype=torch.float32).unsqueeze(1)

# CARREGAR O SCALER EXISTENTE
scaler = None
try:

    scaler = joblib.load(NORMALIZATION_SCALER_FILE)
    print(f"RobustScaler carregado com sucesso de: {NORMALIZATION_SCALER_FILE}")
    print(f"Centro do scaler para desescalar: {scaler.center_[0]:.4f}, Escala: {scaler.scale_[0]:.4f}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro RobustScaler '{NORMALIZATION_SCALER_FILE}' não foi encontrado.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO: Falha ao carregar RobustScaler de {NORMALIZATION_SCALER_FILE}: {e}")
    sys.exit(1)

true_original_target_values = scaler.inverse_transform(df['prot_scaled'].values.reshape(-1, 1)).flatten()

try:
    X_embeddings = np.load(DNABERT_EMBEDDINGS_FILE)
    X_embeddings = torch.tensor(X_embeddings, dtype=torch.float32)
    if X_embeddings.shape[0] != len(df):
        print(f"AVISO: O número de amostras nos embeddings ({X_embeddings.shape[0]}) não corresponde ao número de amostras no CSV ({len(df)}).")
    print(f"Embeddings DNABERT carregados com sucesso de {DNABERT_EMBEDDINGS_FILE}. Dimensão: {X_embeddings.shape}")
except FileNotFoundError:
    print(f"ERRO: O ficheiro de embeddings '{DNABERT_EMBEDDINGS_FILE}' não foi encontrado.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO: Falha ao carregar o ficheiro de embeddings '{DNABERT_EMBEDDINGS_FILE}': {e}")
    sys.exit(1)

print(f"\nDimensão de X (features): {X_embeddings.shape}")
print(f"Dimensão de y (target 'prot_scaled' JÁ ESCALADO): {y_scaled.shape}")


--- A carregar dados e embeddings pré-calculados ---
Dados originais carregados com sucesso de /content/drive/MyDrive/training_data.csv. Dimensão: (9336, 4)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


RobustScaler carregado com sucesso de: /content/drive/MyDrive/scaler_robust.pkl
Centro do scaler para desescalar: 32619.6512, Escala: 93883.9232
Embeddings DNABERT carregados com sucesso de /content/drive/MyDrive/dnabert_embeddings.npy. Dimensão: torch.Size([9336, 768])

Dimensão de X (features): torch.Size([9336, 768])
Dimensão de y (target 'prot_scaled' JÁ ESCALADO): torch.Size([9336, 1])


In [5]:
# --- 4. Definição da Feedforward Neural Network (FNN) ---
class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"FNN a treinar em dispositivo: {device}")

FNN a treinar em dispositivo: cpu


In [6]:
# --- 5. Treinar e Avaliar a FNN com Validação Cruzada ---
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mse_scores_scaled = [] # ALTERADO
mae_scores_scaled = [] # ALTERADO
r2_scores_scaled = [] # ALTERADO
pearson_scores_scaled = [] # ALTERADO

best_fnn_model_overall = None
best_r2_val = -float('inf')

for fold, (train_index, test_index) in enumerate(kf.split(X_embeddings)):
    print(f"\n--- FNN - Fold {fold+1}/{n_splits} ---")

    X_train, X_test = X_embeddings[train_index], X_embeddings[test_index]
    y_train, y_test = y_scaled[train_index], y_scaled[test_index] # y_test é escalado

    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=FNN_BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=FNN_BATCH_SIZE, shuffle=False)

    input_dim = X_embeddings.shape[1]
    model = FNN(input_dim, FNN_HIDDEN_DIM).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=FNN_LEARNING_RATE)

    model.train()
    for epoch in range(FNN_NUM_EPOCHS):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 20 == 0 or epoch == 0 or epoch == FNN_NUM_EPOCHS - 1:
            print(f'  Epoch [{epoch+1}/{FNN_NUM_EPOCHS}], Loss: {total_loss/len(train_loader):.4f}')

    model.eval()
    fold_predictions_scaled = []
    fold_actual_scaled = []
    with torch.no_grad():
        for X_batch_test, y_batch_test in test_loader:
            X_batch_test, y_batch_test = X_batch_test.to(device), y_batch_test.to(device)
            outputs_test = model(X_batch_test)
            fold_predictions_scaled.extend(outputs_test.cpu().numpy().flatten())
            fold_actual_scaled.extend(y_batch_test.cpu().numpy().flatten())

    fold_predictions_scaled = np.array(fold_predictions_scaled)
    fold_actual_scaled = np.array(fold_actual_scaled)
    mse = mean_squared_error(fold_actual_scaled, fold_predictions_scaled)
    mae = mean_absolute_error(fold_actual_scaled, fold_predictions_scaled)
    r2 = r2_score(fold_actual_scaled, fold_predictions_scaled)

    try:
        pearson_corr, _ = pearsonr(fold_actual_scaled, fold_predictions_scaled)
    except ValueError:
        pearson_corr = np.nan

    mse_scores_scaled.append(mse)
    mae_scores_scaled.append(mae)
    r2_scores_scaled.append(r2)
    pearson_scores_scaled.append(pearson_corr)

    print(f"Fold {fold+1} Resultados (escala escalada): MSE={mse:.4f}, MAE={mae:.4f}, R2={r2:.4f}, Pearson={pearson_corr:.4f}")

    if r2 > best_r2_val:
        best_r2_val = r2
        best_fnn_model_overall = model.state_dict()

print("\n--- Resultados Finais da Validação Cruzada (Médias e Desvios Padrão) ---")
final_cv_results = {
    "MSE médio (escala escalada)": f"{np.nanmean(mse_scores_scaled):.4f} +/- {np.nanstd(mse_scores_scaled):.4f}",
    "MAE médio (escala escalada)": f"{np.nanmean(mae_scores_scaled):.4f} +/- {np.nanstd(mae_scores_scaled):.4f}",
    "R2 médio (escala escalada)": f"{np.nanmean(r2_scores_scaled):.4f} +/- {np.nanstd(r2_scores_scaled):.4f}",
    "Correlação de Pearson média (escala escalada)": f"{np.nanmean(pearson_scores_scaled):.4f} +/- {np.nanstd(pearson_scores_scaled):.4f}"
}
for metric, value in final_cv_results.items():
    print(f"{metric}: {value}")


--- FNN - Fold 1/5 ---
  Epoch [1/100], Loss: 0.3678
  Epoch [20/100], Loss: 0.1132
  Epoch [40/100], Loss: 0.0586
  Epoch [60/100], Loss: 0.0285
  Epoch [80/100], Loss: 0.0159
  Epoch [100/100], Loss: 0.0109
Fold 1 Resultados (escala escalada): MSE=0.2150, MAE=0.3370, R2=0.5711, Pearson=0.7712

--- FNN - Fold 2/5 ---
  Epoch [1/100], Loss: 0.3790
  Epoch [20/100], Loss: 0.1236
  Epoch [40/100], Loss: 0.0667
  Epoch [60/100], Loss: 0.0337
  Epoch [80/100], Loss: 0.0200
  Epoch [100/100], Loss: 0.0111
Fold 2 Resultados (escala escalada): MSE=0.1964, MAE=0.3186, R2=0.5740, Pearson=0.7762

--- FNN - Fold 3/5 ---
  Epoch [1/100], Loss: 0.3731
  Epoch [20/100], Loss: 0.1230
  Epoch [40/100], Loss: 0.0626
  Epoch [60/100], Loss: 0.0344
  Epoch [80/100], Loss: 0.0237
  Epoch [100/100], Loss: 0.0122
Fold 3 Resultados (escala escalada): MSE=0.1977, MAE=0.3257, R2=0.5924, Pearson=0.7804

--- FNN - Fold 4/5 ---
  Epoch [1/100], Loss: 0.3782
  Epoch [20/100], Loss: 0.1148
  Epoch [40/100], Loss: 

In [7]:
# --- 6. Avaliação Final no Dataset Completo e Guardar Dados para Gráficos ---

final_model = FNN(input_dim, FNN_HIDDEN_DIM).to(device)
final_criterion = nn.MSELoss()
final_optimizer = optim.Adam(final_model.parameters(), lr=FNN_LEARNING_RATE)

full_dataset = TensorDataset(X_embeddings, y_scaled)
full_loader = DataLoader(full_dataset, batch_size=FNN_BATCH_SIZE, shuffle=True)

final_model.train()
for epoch in range(FNN_NUM_EPOCHS):
    for X_batch, y_batch in full_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        final_optimizer.zero_grad()
        outputs = final_model(X_batch)
        loss = final_criterion(outputs, y_batch)
        loss.backward()
        final_optimizer.step()

final_model.eval()
final_predictions_scaled = []
with torch.no_grad():
    for X_batch_test, _ in DataLoader(TensorDataset(X_embeddings, y_scaled), batch_size=FNN_BATCH_SIZE, shuffle=False):
        X_batch_test = X_batch_test.to(device)
        outputs_test = final_model(X_batch_test)
        final_predictions_scaled.extend(outputs_test.cpu().numpy().flatten())

final_predictions_scaled = np.array(final_predictions_scaled)

# --- Desescalar as previsões para obter valores reais ---
# usando inverse_transform do RobustScaler
final_predictions_original = scaler.inverse_transform(final_predictions_scaled.reshape(-1, 1)).flatten()

global_mse = mean_squared_error(true_original_target_values, final_predictions_original)
global_mae = mean_absolute_error(true_original_target_values, final_predictions_original)
global_r2 = r2_score(true_original_target_values, final_predictions_original)
try:
    pearson_corr_global, _ = pearsonr(true_original_target_values.flatten(), final_predictions_original.flatten())
except ValueError:
    pearson_corr_global = np.nan

global_results = {
    "MSE Global (escala original)": f"{global_mse:.4f}",
    "MAE Global (escala original)": f"{global_mae:.4f}",
    "R2 Global (escala original)": f"{global_r2:.4f}",
    "Pearson Global (escala original)": f"{pearson_corr_global:.4f}"
}
for metric, value in global_results.items():
    print(f"{metric}: {value}")

MSE Global (escala original): 111427405.1398
MAE Global (escala original): 8107.8810
R2 Global (escala original): 0.9734
Pearson Global (escala original): 0.9867


In [8]:
# --- 7. Guardar Resultados e Modelo ---
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
results_filename = os.path.join(RESULTS_OUTPUT_DIR, f"{RESULTS_FILE_PREFIX}_{timestamp}.txt")
predictions_data_filename = os.path.join(RESULTS_OUTPUT_DIR, f"{PREDICTIONS_DATA_PREFIX}_{timestamp}.npy")
model_save_filename = os.path.join(RESULTS_OUTPUT_DIR, f"best_fnn_model_robust_{timestamp}.pth") # ALTERADO: Nome para refletir RobustScaler

print(f"\n--- A guardar resultados em: {results_filename} ---")
with open(results_filename, 'w') as f:
    f.write(f"Resultados da Otimização da FNN com RobustScaler\n") # ALTERADO
    f.write(f"Data e Hora: {timestamp}\n")
    f.write(f"----------------------------------------------------\n\n")

    f.write(f"Hiperparâmetros da FNN:\n")
    f.write(f"  Dimensão Oculta: {FNN_HIDDEN_DIM}\n")
    f.write(f"  Taxa de Aprendizagem: {FNN_LEARNING_RATE}\n")
    f.write(f"  Número de Épocas: {FNN_NUM_EPOCHS}\n")
    f.write(f"  Tamanho do Lote: {FNN_BATCH_SIZE}\n")
    f.write(f"\n")

    f.write(f"Resultados da Validação Cruzada (Médias e Desvios Padrão - ESCALA ESCALADA (RobustScaler)):\n") # ALTERADO
    for metric, value in final_cv_results.items():
        f.write(f"  {metric}: {value}\n")
    f.write(f"\n")

    f.write(f"Métricas no Dataset Completo (Com o modelo final treinado uma vez no dataset inteiro - ESCALA ORIGINAL):\n")
    for metric, value in global_results.items():
        f.write(f"  {metric}: {value}\n")
    f.write(f"\n")
    f.write(f"Informações Adicionais:\n")
    f.write(f"  Número de Amostras: {X_embeddings.shape[0]}\n")
    f.write(f"  Número de Features: {X_embeddings.shape[1]}\n")
    f.write(f"  Tipo de Embeddings: DNABERT\n")
    f.write(f"  Folds de Validação Cruzada: {n_splits}\n")
    f.write(f"  Ficheiro de Embeddings Usado: {os.path.basename(DNABERT_EMBEDDINGS_FILE)}\n")
    f.write(f"  Ficheiro de Modelo FNN Guardado: {os.path.basename(model_save_filename)}\n")
    f.write(f"  Ficheiro de Dados de Previsões Guardado: {os.path.basename(predictions_data_filename)}\n")
    f.write(f"  Ficheiro do Scaler Guardado: {os.path.basename(NORMALIZATION_SCALER_FILE)}\n") # ALTERADO


print(f"Resultados guardados com sucesso em {results_filename}.")

data_for_plotting = np.vstack((true_original_target_values.flatten(), final_predictions_original)).T # ALTERADO
np.save(predictions_data_filename, data_for_plotting)
print(f"Valores reais (escala original) e previstos (escala original) guardados em {predictions_data_filename} para análise de gráficos.")

torch.save(final_model.state_dict(), model_save_filename)
print(f"Melhor modelo FNN guardado em: {model_save_filename}.")

print("\nScript de treino e avaliação da FNN concluído.")


--- A guardar resultados em: /content/drive/MyDrive/fnn_robust_results_20250620_224238.txt ---
Resultados guardados com sucesso em /content/drive/MyDrive/fnn_robust_results_20250620_224238.txt.
Valores reais (escala original) e previstos (escala original) guardados em /content/drive/MyDrive/fnn_robust_predictions_data_20250620_224238.npy para análise de gráficos.
Melhor modelo FNN guardado em: /content/drive/MyDrive/best_fnn_model_robust_20250620_224238.pth.

Script de treino e avaliação da FNN concluído.
